# HyperSpectral YOLO Pipeline

## imports

In [ ]:

from pathlib import Path
import pandas as pd
from ultralytics import YOLO
import os
import cv2
import matlab.engine
import numpy as np
from IPython.display import Image
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import matplotlib.patches as patches
from PIL import Image
import shutil

import warnings
warnings.filterwarnings("ignore")
from ultralytics import YOLO, checks
import torch
checks()

Ultralytics 8.3.59  Python-3.12.6 torch-2.6.0.dev20241216+cu124 CUDA:0 (NVIDIA GeForce RTX 2060, 6144MiB)
Setup complete  (12 CPUs, 31.9 GB RAM, 790.0/930.9 GB disk)


In [3]:
eng = matlab.engine.start_matlab()
eng.quit()

In [4]:


def detect_anomalies(image_path, dictionary_path='hs_dictionaries.mat', numb_pca_components =4,num_ica_components = 4, ICA_max_iterations = 1000, similarity_threshold =1, opts_lambda = 0.4, opts_max_iter = 100, lambda_ = 0.01, overlap_threshold = 0.2, min_area = 50,max_area=3200, min_anomalous_pixels = 20, spar =28):
    # Start MATLAB engine
    

    # Convert RGB to hyperspectral (rec_hs is returned directly)
    rec_hs = eng.convertToHyperspectral(image_path, dictionary_path, spar)

    # Perform anomaly detection
    bounding_boxes = eng.anomalyDetection(
        rec_hs,
        numb_pca_components,
        num_ica_components,
        ICA_max_iterations,
        similarity_threshold,
        opts_lambda,
        opts_max_iter,
        lambda_,
        overlap_threshold,
        min_area,
        max_area,
        min_anomalous_pixels
        
        
    )
    # Convert MATLAB bounding boxes to Python list
    bounding_boxes = [list(bbox) for bbox in bounding_boxes]

    return bounding_boxes


## running the models

### Detection using YOLOv8 only

In [ ]:

# Correct path to the trained weights

# Correct path to the trained weights
model = YOLO(r"yolov8m-drone.pt")

# Path to testing folder
test_images_dir = r"C:\Users\michael\Desktop\f_proj\images"  # Folder containing test images
results_output_dir = r"C:\Users\michael\Desktop\f_proj\Yolores"  # Directory to save results
os.makedirs(results_output_dir, exist_ok=True)

# Run inference on the testing folder
results = model.predict(
    source=test_images_dir,  # Path to test images
    save=True, 
    conf=0.25,              # default confidence threshold
    save_txt=True,           # Save predictions in text format
    save_conf=True,            # Save confidence scores
    project=results_output_dir,  # Save results in this folder
    imgsz=640                # Image size
)

# Print summary of results
print("Summary of Detection Results:")
print(f"Number of images tested: {len(results)}")
print(f"Results saved in: {results_output_dir}")

# Display per-image metrics
for result in results:
    print(f"Image: {result.path}, Detections: {len(result.boxes)}")



image 1/596 C:\Users\michael\Desktop\f_proj\birdrone_research_data\valid\images\-de-pajaros-volando-sobre-una-zona-suburbana-de-san-petersburgo-rusia-s1dm2t_jpg.rf.90e8cfeb7ba2689f7398d8333e41efdc.jpg: 640x640 203 0s, 97 1s, 23.9ms
image 2/596 C:\Users\michael\Desktop\f_proj\birdrone_research_data\valid\images\000000000520_jpg.rf.a9f624ed59d8e7874d390be53bfbcfe1.jpg: 640x640 151 0s, 149 1s, 23.9ms
image 3/596 C:\Users\michael\Desktop\f_proj\birdrone_research_data\valid\images\107_JPEG_jpg.rf.b8dec912dd7e106d919f6afedecdac02.jpg: 640x640 241 0s, 59 1s, 22.9ms
image 4/596 C:\Users\michael\Desktop\f_proj\birdrone_research_data\valid\images\10_JPEG_jpg.rf.8f8623900e781ba74d25b74b6242215b.jpg: 640x640 271 0s, 29 1s, 29.0ms
image 5/596 C:\Users\michael\Desktop\f_proj\birdrone_research_data\valid\images\123_JPEG_jpg.rf.d6c4335f294c1e43997e0f3d7794d28e.jpg: 640x640 112 0s, 188 1s, 25.9ms
image 6/596 C:\Users\michael\Desktop\f_proj\birdrone_research_data\valid\images\127_JPEG_jpg.rf.9b76d3db49

### Detection using our Pipeline

In [ ]:
def compute_iou(boxA, boxB):
    # boxA and boxB: (x1, y1, x2, y2)
    xA = max(boxA[0], boxB[0])
    yA = max(boxA[1], boxB[1])
    xB = min(boxA[2], boxB[2])
    yB = min(boxA[3], boxB[3])
    interArea = max(0, xB - xA + 1) * max(0, yB - yA + 1)
    boxAArea = (boxA[2] - boxA[0] + 1) * (boxA[3] - boxA[1] + 1)
    boxBArea = (boxB[2] - boxB[0] + 1) * (boxB[3] - boxB[1] + 1)
    iou = interArea / float(boxAArea + boxBArea - interArea)
    return iou

In [ ]:

def is_contained(boxA, boxB):
    # Checks if boxA is at least 50% contained in boxB or vice versa.
    xA = max(boxA[0], boxB[0])
    yA = max(boxA[1], boxB[1])
    xB = min(boxA[2], boxB[2])
    yB = min(boxA[3], boxB[3])
    interArea = max(0, xB - xA + 1) * max(0, yB - yA + 1)
    areaA = (boxA[2] - boxA[0] + 1) * (boxA[3] - boxA[1] + 1)
    areaB = (boxB[2] - boxB[0] + 1) * (boxB[3] - boxB[1] + 1)
    if (interArea / areaA >= 0.5) or (interArea / areaB >= 0.5):
        return True
    return False

In [ ]:
def cross_class_nms(detections, iou_thresh=0.5):
    """
    A simple cross-class NMS that keeps only the highest-confidence detection
    among overlapping boxes (IoU >= iou_thresh).
    Each detection is (class_id, confidence, x1, y1, x2, y2).
    """
    detections = sorted(detections, key=lambda d: d[1], reverse=True)
    final_dets = []
    used = [False] * len(detections)
    for i in range(len(detections)):
        if used[i]:
            continue
        final_dets.append(detections[i])
        for j in range(i + 1, len(detections)):
            if used[j]:
                continue
            iou_val = compute_iou(detections[i][2:6], detections[j][2:6])
            if iou_val >= iou_thresh:
                used[j] = True
    return final_dets

In [ ]:

def merge_bounding_boxes(bboxes, proximity_threshold=20, max_area=30000):
    """
    Merge bounding boxes (list of (x, y, w, h)) if their centers are within proximity_threshold.
    Merge is done only if the new area <= max_area.
    Returns merged boxes as (x, y, w, h).
    """
    if not bboxes:
        return []
    merged = []
    used = [False] * len(bboxes)
    for i in range(len(bboxes)):
        if used[i]:
            continue
        x, y, w, h = bboxes[i]
        merged_box = [x, y, x + w, y + h]
        used[i] = True
        for j in range(i + 1, len(bboxes)):
            if used[j]:
                continue
            x2, y2, w2, h2 = bboxes[j]
            centerA = ((merged_box[0] + merged_box[2]) / 2, (merged_box[1] + merged_box[3]) / 2)
            centerB = (x2 + w2 / 2, y2 + h2 / 2)
            dist = ((centerA[0] - centerB[0])**2 + (centerA[1] - centerB[1])**2)**0.5
            if dist <= proximity_threshold:
                new_x1 = min(merged_box[0], x2)
                new_y1 = min(merged_box[1], y2)
                new_x2 = max(merged_box[2], x2 + w2)
                new_y2 = max(merged_box[3], y2 + h2)
                new_w = new_x2 - new_x1
                new_h = new_y2 - new_y1
                if new_w * new_h <= max_area:
                    merged_box = [new_x1, new_y1, new_x2, new_y2]
                    used[j] = True
        merged.append((merged_box[0], merged_box[1], merged_box[2]-merged_box[0], merged_box[3]-merged_box[1]))
    return merged

In [ ]:

def deduplicate_pipeline_dets(detections, overlap_thresh=0.5):
    """
    Deduplicate detections (list of (class, conf, x1, y1, x2, y2)) by discarding duplicates
    if IoU >= overlap_thresh or one box is contained in another.
    """
    if not detections:
        return []
    detections = sorted(detections, key=lambda d: d[1], reverse=True)
    final_dets = []
    used = [False] * len(detections)
    for i in range(len(detections)):
        if used[i]:
            continue
        det_i = detections[i]
        final_dets.append(det_i)
        for j in range(i+1, len(detections)):
            if used[j]:
                continue
            if compute_iou(det_i[2:6], detections[j][2:6]) >= overlap_thresh or is_contained(det_i[2:6], detections[j][2:6]):
                used[j] = True
    return final_dets

In [ ]:
# *** Vectorized IoU Computation for Re-adding Non-Drone Detections ***
def compute_iou_vectorized(boxes1, boxes2):
    """
    Compute the IoU matrix between two sets of boxes.
    boxes1: numpy array of shape (N, 4)
    boxes2: numpy array of shape (M, 4)
    Returns an array of shape (N, M) with IoU values.
    """
    boxes1 = np.array(boxes1)  # shape (N, 4)
    boxes2 = np.array(boxes2)  # shape (M, 4)
    
    # Compute intersection coordinates
    xA = np.maximum(boxes1[:, None, 0], boxes2[None, :, 0])
    yA = np.maximum(boxes1[:, None, 1], boxes2[None, :, 1])
    xB = np.minimum(boxes1[:, None, 2], boxes2[None, :, 2])
    yB = np.minimum(boxes1[:, None, 3], boxes2[None, :, 3])
    
    interWidth = np.maximum(0, xB - xA + 1)
    interHeight = np.maximum(0, yB - yA + 1)
    interArea = interWidth * interHeight
    
    area1 = (boxes1[:, 2] - boxes1[:, 0] + 1) * (boxes1[:, 3] - boxes1[:, 1] + 1)
    area2 = (boxes2[:, 2] - boxes2[:, 0] + 1) * (boxes2[:, 3] - boxes2[:, 1] + 1)
    
    unionArea = area1[:, None] + area2[None, :] - interArea
    iou_matrix = interArea / unionArea
    return iou_matrix

In [ ]:


# This threshold is used to filter out low-confidence detections in the anomaly detection pipeline.
CONF_THRESHOLD = 0.28
YOLO_OVERLAP_THRESH = 0.0  # for discarding pipeline boxes based on overlap with full-image YOLO boxes
PIPELINE_DEDUP_THRESH = 0.5  # for deduplication in pipeline NMS

# Main processing
paddings = [200]
model = YOLO(r"yolov8m-drone.pt")
eng = matlab.engine.start_matlab()
max_size = 31000  # Maximum area for bounding boxes`
min_area = 300  # Minimum area for bounding boxes
for padding in paddings:
    test_images_dir = r"C:\Users\michael\Desktop\f_proj\output_images\test2"
    results_output_dir = fr"C:\Users\michael\Desktop\f_proj\300min"  # Directory to save results
    os.makedirs(results_output_dir, exist_ok=True)
    labels_output_dir = os.path.join(results_output_dir, "labels")
    os.makedirs(labels_output_dir, exist_ok=True)
    IMG_EXTS = (".jpg", ".jpeg", ".png", ".bmp", ".tif", ".tiff",
            ".webp", ".dng", ".pfm", ".mpo")          # skip non-image files

    test_images = [
        os.path.join(test_images_dir, fname)
        for fname in os.listdir(test_images_dir)
        if fname.lower().endswith(IMG_EXTS)
    ]
    
    for image_path in test_images:
        print(f"Processing: {image_path}")
        results = model.predict(source=image_path, imgsz=640, save=False)
        yolo_labels = []
        detections = results[0].boxes  # YOLO detections on the full image
        original_image = cv2.imread(image_path)
        
        yolo_detections = []
        non_drone_dets = []
        if detections is not None and len(detections) > 0:
            for box in detections:
                x1, y1, x2, y2 = map(int, box.xyxy[0])
                class_id = int(box.cls[0])
                confidence = float(box.conf[0])
                print(f"Full-image YOLO: Class {class_id}, Conf {confidence:.2f}, BBox ({x1}, {y1}, {x2}, {y2})")
                if class_id == 0:
                    yolo_detections.append((x1, y1, x2, y2, class_id, confidence))
                    # Save YOLO detections with "y" prefix
                    x_center = (x1 + x2) / 2 / original_image.shape[1]
                    y_center = (y1 + y2) / 2 / original_image.shape[0]
                    bbox_width = (x2 - x1) / original_image.shape[1]
                    bbox_height = (y2 - y1) / original_image.shape[0]
                    yolo_labels.append(f"y {class_id} {x_center:.6f} {y_center:.6f} {bbox_width:.6f} {bbox_height:.6f} {confidence:.6f}")
                else:
                    non_drone_dets.append((x1, y1, x2, y2, class_id, confidence))
            print(f"Drones (class=0) detected in {image_path} by YOLO: {len(yolo_detections)}")
        else:
            print(f"No YOLO detections in {image_path}.")
        
        # Draw YOLO drone boxes
        for (x1, y1, x2, y2, cls_id, conf) in yolo_detections:
            cv2.rectangle(original_image, (x1, y1), (x2, y2), color=(0, 255, 0), thickness=2)
        
        # 2) Run anomaly detection pipeline
        try:
            bounding_boxes = detect_anomalies(image_path, max_area=max_size, numb_pca_components=4)
            merged_boxes = merge_bounding_boxes(bounding_boxes, proximity_threshold=20, max_area=max_size)
            merged_boxes = merge_bounding_boxes(merged_boxes, proximity_threshold=20, max_area=max_size)
            
            pipeline_detections_global = []
            # For each merged anomaly bounding box, run YOLO on the ROI
            for bbox in merged_boxes:
                x, y, w, h = map(int, bbox)
                # Compute padded ROI (before zooming)
                x_padded = max(x - padding, 0)
                y_padded = max(y - padding, 0)
                w_padded = min(w + 2 * padding, original_image.shape[1] - x_padded)
                h_padded = min(h + 2 * padding, original_image.shape[0] - y_padded)
                
                # This ensures that the zoomed ROI remains centered around the original box
                orig_center_x = x + w / 2
                orig_center_y = y + h / 2
                padded_center_x = x_padded + w_padded / 2
                padded_center_y = y_padded + h_padded / 2
                offset_x = orig_center_x - padded_center_x
                offset_y = orig_center_y - padded_center_y
                x_padded = max(int(x_padded + offset_x), 0)
                y_padded = max(int(y_padded + offset_y), 0)
                # Recalculate ROI width and height to remain within image bounds
                w_padded = min(w + 2 * padding, original_image.shape[1] - x_padded)
                h_padded = min(h + 2 * padding, original_image.shape[0] - y_padded)
                print(f"ROI before zooming: (x={x_padded}, y={y_padded}, w={w_padded}, h={h_padded})")
                
                # zoom factor based on padded ROI size
                zoom_factor = 3 if (w_padded < 200 or h_padded < 200) else 2
                if zoom_factor > 2:
                    roi = original_image[y_padded:y_padded+h_padded, x_padded:x_padded+w_padded]
                    roi_zoomed = cv2.resize(roi, (roi.shape[1]*zoom_factor, roi.shape[0]*zoom_factor))
                    roi_for_yolo = cv2.resize(roi_zoomed, (640, 640))
                    effective_w = w_padded * zoom_factor
                    effective_h = h_padded * zoom_factor
                    print(f"Zooming applied with factor {zoom_factor}.")
                else:
                    roi = original_image[y_padded:y_padded+h_padded, x_padded:x_padded+w_padded]
                    roi_for_yolo = cv2.resize(roi, (640, 640))
                    effective_w = w_padded
                    effective_h = h_padded
                # Draw the padded ROI on the original image
                roi_results = model.predict(source=roi_for_yolo, conf=CONF_THRESHOLD, save=False)
                roi_detections = roi_results[0].boxes
                if roi_detections is not None and len(roi_detections) > 0:
                    print(f"Anomaly ROI from {image_path}: (x={x_padded}, y={y_padded}, w={w_padded}, h={h_padded})")
                    roi_dets_list = []
                    for roi_box in roi_detections:
                        x1_r, y1_r, x2_r, y2_r = map(int, roi_box.xyxy[0])
                        conf_r = float(roi_box.conf[0])
                        cls_r = int(roi_box.cls[0])
                        print(f"  ROI detection: Class {cls_r}, Conf {conf_r:.2f}, BBox ({x1_r}, {y1_r}, {x2_r}, {y2_r})")
                        roi_dets_list.append((cls_r, conf_r, x1_r, y1_r, x2_r, y2_r))
                    filtered_dets = cross_class_nms(roi_dets_list, iou_thresh=0.25)
                    # Filter out detections that are too small or have low confidence
                    for (cls_r, conf_r, x1_r, y1_r, x2_r, y2_r) in filtered_dets:
                        if cls_r == 0:
                            x1_abs = x1_r * effective_w // 640 + x_padded
                            x2_abs = x2_r * effective_w // 640 + x_padded
                            y1_abs = y1_r * effective_h // 640 + y_padded
                            y2_abs = y2_r * effective_h // 640 + y_padded
                            pipeline_box = (x1_abs, y1_abs, x2_abs, y2_abs)
                            discard_pipeline = False
                            for yd in yolo_detections:
                                if compute_iou(pipeline_box, yd[:4]) > YOLO_OVERLAP_THRESH or is_contained(pipeline_box, yd[:4]):
                                    discard_pipeline = True
                                    break
                            if not discard_pipeline:
                                # Check aspect ratio constraints
                                box_width = x2_abs - x1_abs
                                box_height = y2_abs - y1_abs
                                ratio = box_width / box_height if box_height > 0 else 0
                                if ratio < 0.17 or ratio > 1.52:
                                    print(f"Discarding pipeline detection due to aspect ratio {ratio:.2f}")
                                    continue
                                print(f"  -> Pipeline DRONE candidate: Conf={conf_r:.2f}, Box=({x1_abs}, {y1_abs}, {x2_abs}, {y2_abs})")
                                pipeline_detections_global.append((cls_r, conf_r, x1_abs, y1_abs, x2_abs, y2_abs))
                else:
                    print(f"No ROI detections in ROI from {image_path}.")
            #  Deduplicate Pipeline Detections 
            dedup_pipeline_dets = deduplicate_pipeline_dets(pipeline_detections_global, overlap_thresh=PIPELINE_DEDUP_THRESH)
            
            for (cls_r, conf_r, x1_abs, y1_abs, x2_abs, y2_abs) in dedup_pipeline_dets:
                final_area = (x2_abs - x1_abs) * (y2_abs - y1_abs)
                if final_area > max_size or final_area < min_area:
                    print(f"Discarding pipeline detection with area {final_area} pixels (exceeds limit)")
                    continue
                print(f"  -> Final Pipeline DRONE kept: Conf={conf_r:.2f}, Box=({x1_abs}, {y1_abs}, {x2_abs}, {y2_abs})")
                x_center = (x1_abs + x2_abs) / 2 / original_image.shape[1]
                y_center = (y1_abs + y2_abs) / 2 / original_image.shape[0]
                bbox_width = (x2_abs - x1_abs) / original_image.shape[1]
                bbox_height = (y2_abs - y1_abs) / original_image.shape[0]
                yolo_labels.append(f"p {cls_r} {x_center:.6f} {y_center:.6f} {bbox_width:.6f} {bbox_height:.6f} {conf_r:.6f}")
                cv2.rectangle(original_image, (x1_abs, y1_abs), (x2_abs, y2_abs), color=(0, 255, 0), thickness=2)
            
            # Vectorized Re-addition of Non-Drone YOLO Detections
            # Convert final pipeline boxes (from yolo_labels) to absolute xyxy format.
            final_pipeline_boxes = []
            for lbl in yolo_labels:
                parts = lbl.strip().split()
                prefix, cls_id, xc, yc, w, h, conf_lbl = parts
                xc, yc, w, h = map(float, (xc, yc, w, h))
                conf_lbl = float(conf_lbl)
                x_min = (xc - w/2) * original_image.shape[1]
                y_min = (yc - h/2) * original_image.shape[0]
                x_max = (xc + w/2) * original_image.shape[1]
                y_max = (yc + h/2) * original_image.shape[0]
                final_pipeline_boxes.append([x_min, y_min, x_max, y_max])
            final_pipeline_boxes = np.array(final_pipeline_boxes) if final_pipeline_boxes else np.empty((0,4))
            
            # Convert non_drone_dets to numpy array (only xyxy part)
            non_drone_boxes = np.array([det[:4] for det in non_drone_dets]) if non_drone_dets else np.empty((0,4))
            if non_drone_boxes.size > 0 and final_pipeline_boxes.size > 0:
                iou_matrix = compute_iou_vectorized(non_drone_boxes, final_pipeline_boxes)
                max_iou = np.max(iou_matrix, axis=1)
            elif non_drone_boxes.size > 0:
                max_iou = np.zeros(len(non_drone_dets))
            else:
                max_iou = np.array([])
            overlap_threshold = 0.5
            # Re-add non-drone detections that do not overlap significantly with pipeline boxes
            indices_to_readd = np.where(max_iou < overlap_threshold)[0]
            for idx in indices_to_readd:
                nx1, ny1, nx2, ny2, ncls, nconf = non_drone_dets[idx]
                print(f"Re-adding YOLO non-drone box (class={ncls}) with no significant overlap: ({nx1}, {ny1}, {nx2}, {ny2})")
                cv2.rectangle(original_image, (nx1, ny1), (nx2, ny2), color=(255, 0, 0), thickness=2)
                x_center = (nx1 + nx2) / 2 / original_image.shape[1]
                y_center = (ny1 + ny2) / 2 / original_image.shape[0]
                bbox_width = (nx2 - nx1) / original_image.shape[1]
                bbox_height = (ny2 - ny1) / original_image.shape[0]
                yolo_labels.append(f"y {ncls} {x_center:.6f} {y_center:.6f} {bbox_width:.6f} {bbox_height:.6f} {nconf:.6f}")
            
            output_path = os.path.join(results_output_dir, os.path.basename(image_path))
            cv2.imwrite(output_path, original_image)
            
            label_path = os.path.join(labels_output_dir, os.path.splitext(os.path.basename(image_path))[0] + ".txt")
            print("Final label set:", yolo_labels)
            with open(label_path, "w") as f:
                f.write("\n".join(yolo_labels))
        # If the pipeline fails, fall back to YOLO-only detections        
        except Exception as e:
            print(f"Pipeline failed, falling back to YOLO-only for {image_path}: {e}")

            # 1) re-add every original YOLO detection (both drone & bird)
            for x1, y1, x2, y2, cls, conf in non_drone_dets:
                xc = (x1 + x2) / 2 / original_image.shape[1]
                yc = (y1 + y2) / 2 / original_image.shape[0]
                w  = (x2 - x1) / original_image.shape[1]
                h  = (y2 - y1) / original_image.shape[0]
                yolo_labels.append(
                    f"y {cls} {xc:.6f} {yc:.6f} {w:.6f} {h:.6f} {conf:.6f}"
                )

            # 2) write them out and skip the rest of the loop
            label_path = os.path.join(
                labels_output_dir,
                os.path.splitext(os.path.basename(image_path))[0] + ".txt"
            )
            with open(label_path, "w") as f:
                f.write("\n".join(yolo_labels))

            continue
                
eng.quit()
print("\nPipeline completed.")


Processing: C:\Users\michael\Desktop\f_proj\output_images\test2\10.jpg

image 1/1 C:\Users\michael\Desktop\f_proj\output_images\test2\10.jpg: 480x640 1 0, 114.7ms
Speed: 2.0ms preprocess, 114.7ms inference, 18.9ms postprocess per image at shape (1, 3, 480, 640)
Full-image YOLO: Class 0, Conf 0.88, BBox (171, 147, 329, 291)
Drones (class=0) detected in C:\Users\michael\Desktop\f_proj\output_images\test2\10.jpg by YOLO: 1
ROI before zooming: (x=0, y=0, w=476, h=472)

0: 640x640 (no detections), 41.3ms
Speed: 4.0ms preprocess, 41.3ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)
No ROI detections in ROI from C:\Users\michael\Desktop\f_proj\output_images\test2\10.jpg.
Final label set: ['y 0 0.390625 0.456250 0.246875 0.300000 0.875813']
Processing: C:\Users\michael\Desktop\f_proj\output_images\test2\1030.jpg

image 1/1 C:\Users\michael\Desktop\f_proj\output_images\test2\1030.jpg: 480x640 (no detections), 33.9ms
Speed: 2.0ms preprocess, 33.9ms inference, 0.0ms postprocess

## Metric comparison

###  XML format Gt

In [ ]:
import os
from pathlib import Path
import xml.etree.ElementTree as ET
from sklearn.metrics import precision_score, recall_score, f1_score
import numpy as np
from statsmodels.stats.proportion import proportion_confint

#### Basic Metrics

In [ ]:

def load_xml_ground_truths(xml_folder, img_width, img_height, class_mapping=None):
    """
    Load ground truth annotations stored in XML files (e.g., Pascal VOC format).
    
    Each XML file is expected to contain one or more <object> elements with:
      <name> for the class (e.g., "drone")
      <bndbox> containing: <xmin>, <ymin>, <xmax>, <ymax>
    
    The bounding box is converted to YOLO format (normalized):
      x_center = (xmin + xmax) / 2 / img_width
      y_center = (ymin + ymax) / 2 / img_height
      width    = (xmax - xmin) / img_width
      height   = (ymax - ymin) / img_height
    
    Each returned bounding box is formatted as:
      [prefix, class_id, x_center, y_center, width, height]
    where prefix is set to None (as ground truth typically does not use a prefix).
    
    Args:
      xml_folder (str): Folder containing the XML annotation files.
      img_width (int): Width of the image in pixels.
      img_height (int): Height of the image in pixels.
      class_mapping (dict): Dictionary mapping object names (strings) to class_ids (ints).
                            For example: {"drone": 0}. If None, defaults to {"drone": 0}.
    
    Returns:
      dict: { image_name: list of bounding boxes } where image_name is the XML file stem.
    """
    if class_mapping is None:
        class_mapping = {"drone": 0}
    
    labels = {}
    for xml_file in Path(xml_folder).glob("*.xml"):
        tree = ET.parse(xml_file)
        root = tree.getroot()
        
        image_name = xml_file.stem  
        
        bboxes = []
        for obj in root.findall("object"):
            name = obj.find("name").text.strip()
            if name not in class_mapping:
                # Skip objects not in our mapping
                continue
            class_id = class_mapping[name]
            
            bndbox = obj.find("bndbox")
            xmin = float(bndbox.find("xmin").text)
            ymin = float(bndbox.find("ymin").text)
            xmax = float(bndbox.find("xmax").text)
            ymax = float(bndbox.find("ymax").text)
            
            # Convert to normalized YOLO format
            x_center = ((xmin + xmax) / 2) / img_width
            y_center = ((ymin + ymax) / 2) / img_height
            width = (xmax - xmin) / img_width
            height = (ymax - ymin) / img_height
            
            # ground truths do not have a prefix; we set it to None
            bboxes.append([None, class_id, x_center, y_center, width, height])
        
        labels[image_name] = bboxes
    return labels

In [ ]:
def load_yolo_labels(label_folder, normalize=False, img_width=480, img_height=640):
    """
    Load YOLO-format labels from a folder. Lines can be:
      5 tokens:  class x_center y_center width height
      6 tokens:  class x_center y_center width height conf
      6 tokens:  prefix class x_center y_center width height
      7 tokens:  prefix class x_center y_center width height conf

    We detect numeric vs. non-numeric first token to distinguish.
    If normalize=True and any coord>1, we divide by (img_width,img_height).
    Returns dict of image_name -> [ [prefix, class_id, x_c,y_c,w,h], … ]
    """
    labels = {}
    for label_file in Path(label_folder).glob("*.txt"):
        image_name = label_file.stem
        with open(label_file) as f:
            bboxes = []
            for line in f:
                parts = line.strip().split()
                if not parts:
                    continue

                # drop trailing confidence if present
                if len(parts) in (6,7):
                    # if 7 tokens, always drop the last (conf)
                    # if 6 tokens, might be either no-prefix+conf or prefix+coords
                    if len(parts) == 7:
                        parts = parts[:6]
                    else:  # len == 6
                        # try parsing the first token as float:
                        try:
                            float(parts[0])
                            # it is numeric ⇒ [class,x,y,w,h,conf]; drop conf
                            parts = parts[:5]
                        except ValueError:
                            # non-numeric ⇒ [prefix,class,x,y,w,h]; keep all six
                            pass

                # now parts is length 5 or 6
                if len(parts) == 5:
                    prefix = None
                    class_id, x_c, y_c, w, h = map(float, parts)
                elif len(parts) == 6:
                    prefix = parts[0]
                    class_id, x_c, y_c, w, h = map(float, parts[1:])
                else:
                    # unexpected format
                    print(f"Skipping bad line ({len(parts)} tokens): {line.strip()}")
                    continue

                # optional renormalization
                if normalize and any(val > 1 for val in (x_c, y_c, w, h)):
                    x_c /= img_width
                    y_c /= img_height
                    w   /= img_width
                    h   /= img_height

                bboxes.append([prefix, class_id, x_c, y_c, w, h])

        labels[image_name] = bboxes
    return labels


In [ ]:
def yolo_to_xyxy(bbox, img_width, img_height):
    """
    Convert from [prefix, class_id, x_center, y_center, w, h]
    or [class_id, x_center, y_center, w, h]
    to [x_min, y_min, x_max, y_max].
    """
    if len(bbox) == 5:
        # old: [class_id, x_center, y_center, width, height]
        class_id, x_center, y_center, width, height = bbox
    else:
        # new: [prefix, class_id, x_center, y_center, width, height]
        _, class_id, x_center, y_center, width, height = bbox
    
    x_min = (x_center - width / 2) * img_width
    y_min = (y_center - height / 2) * img_height
    x_max = (x_center + width / 2) * img_width
    y_max = (y_center + height / 2) * img_height
    return [x_min, y_min, x_max, y_max]

In [ ]:
def calculate_iou(box1, box2):
    x_min = max(box1[0], box2[0])
    y_min = max(box1[1], box2[1])
    x_max = min(box1[2], box2[2])
    y_max = min(box1[3], box2[3])
    
    intersection = max(0, x_max - x_min) * max(0, y_max - y_min)
    area_box1 = (box1[2] - box1[0]) * (box1[3] - box1[1])
    area_box2 = (box2[2] - box2[0]) * (box2[3] - box2[1])
    
    union = area_box1 + area_box2 - intersection
    return intersection / union if union > 0 else 0

In [ ]:
def categorize_images(labels, img_width, img_height):
    """
    Categorize images based on bounding box area ratio:
      <=1%, 1%-5%, >5%
    Each bounding box is either [prefix, class_id, x_c, y_c, w, h]
    or [class_id, x_c, y_c, w, h].
    """
    categories = {"<=1%": [], "1%-5%": [], ">5%": []}
    
    for image_name, bboxes in labels.items():
        for bbox in bboxes:
            if len(bbox) == 5:
                class_id, x_c, y_c, w, h = bbox
            else:
                prefix, class_id, x_c, y_c, w, h = bbox
            
            width_pixels  = w * img_width
            height_pixels = h * img_height
            area_ratio = (width_pixels * height_pixels) / (img_width * img_height)

            if area_ratio <= 0.01:
                categories["<=1%"].append(image_name)
            elif 0.01 < area_ratio <= 0.05:
                categories["1%-5%"].append(image_name)
            else:
                categories[">5%"].append(image_name)

    return categories

In [ ]:



def evaluate_detections(ground_truths, detections, img_width, img_height,
                        iou_threshold=0.5, category_images=None):
    """
    Now also returns:
      - 'IoU Count': number of matched bboxes
      - 'IoU Sum':   sum of all IoU values
    Metrics are computed *only* on class_id == 0 (drones).
    """

   
    def is_drone(box):
       
        return (box[1] if len(box)==6 else box[0]) == 0

    # keep only drone GTs
    ground_truths = {
        img: [gt for gt in gts if is_drone(gt)]
        for img, gts in ground_truths.items()
    }
    # keep only drone preds
    detections = {
        img: [p for p in preds if is_drone(p)]
        for img, preds in detections.items()
    }

    
    if category_images is not None:
        ground_truths = {k: v for k, v in ground_truths.items() if k in category_images}
        detections   = {k: v for k, v in detections.items()   if k in category_images}


    tp = fp = fn = 0
    ious = []

    for image_name, gt_bboxes in ground_truths.items():
        preds = detections.get(image_name, [])

        # dedupe by ignoring prefix
        seen = set(); unique_preds = []
        for p in preds:
            key = tuple(p[1:]) if len(p)==6 else tuple(p)
            if key not in seen:
                seen.add(key)
                unique_preds.append(p)

        matched = set()
        for pred in unique_preds:
            pred_xy = yolo_to_xyxy(pred, img_width, img_height)
            found = False
            for i, gt in enumerate(gt_bboxes):
                if i in matched: continue
                gt_xy = yolo_to_xyxy(gt, img_width, img_height)
                iou = calculate_iou(pred_xy, gt_xy)
                if iou >= iou_threshold:
                    # since both are drones, this is always a true positive
                    tp += 1
                    matched.add(i)
                    ious.append(iou)
                    found = True
                    break
            if not found:
                # predicted a drone where none existed → false positive
                fp += 1

        # any drone GT left unmatched → false negatives
        fn += (len(gt_bboxes) - len(matched))

    precision = tp / (tp+fp) if tp+fp>0 else 0
    recall    = tp / (tp+fn) if tp+fn>0 else 0
    f1        = 2*precision*recall / (precision+recall) if precision+recall>0 else 0
    mean_iou  = np.mean(ious) if ious else 0

    alpha = 0.05
    prec_ci_low, prec_ci_upp  = proportion_confint(tp, tp+fp, alpha=alpha, method='wilson')
    rec_ci_low,  rec_ci_upp   = proportion_confint(tp, tp+fn, alpha=alpha, method='wilson')

    return {
        "Precision":       precision,
        "Precision CI":    (prec_ci_low, prec_ci_upp),
        "Recall":          recall,
        "Recall CI":       (rec_ci_low, rec_ci_upp),
        "F1-Score":        f1,
        "Mean IoU":        mean_iou,
        "IoU Count":       len(ious),
        "IoU Sum":         sum(ious),
        "True Positives":  tp,
        "False Positives": fp,
        "False Negatives": fn,
    }

In [ ]:

def combine_metrics(m1, m2):
    tp = m1["True Positives"]   + m2["True Positives"]
    fp = m1["False Positives"]  + m2["False Positives"]
    fn = m1["False Negatives"]  + m2["False Negatives"]
    prec = tp / (tp+fp) if tp+fp>0 else 0
    rec  = tp / (tp+fn) if tp+fn>0 else 0
    f1   = 2*prec*rec / (prec+rec) if prec+rec>0 else 0
    # weighted mean IoU
    sum_iou   = m1["IoU Sum"] + m2["IoU Sum"]
    count_iou = m1["IoU Count"] + m2["IoU Count"]
    mean_iou  = sum_iou / count_iou if count_iou>0 else 0
     # 95% Wilson confidence intervals
    alpha = 0.05
    prec_ci_low,  prec_ci_upp  = proportion_confint(tp, tp+fp, alpha=alpha, method='wilson')
    rec_ci_low,   rec_ci_upp   = proportion_confint(tp, tp+fn, alpha=alpha, method='wilson')
    return {
        "Precision":          prec,
        "Precision CI":       (prec_ci_low, prec_ci_upp),
        "Recall":             rec,
        "Recall CI":          (rec_ci_low, rec_ci_upp),
        "F1-Score":           f1,
        "Mean IoU":           mean_iou,
        "True Positives":     tp,
        "False Positives":    fp,
        "False Negatives":    fn,
    }

In [ ]:
# --- Dataset 1 (your original test set: XML GT, 640×480) ---
gt_folder1   = r"C:\Users\michael\Desktop\f_proj\output_xml\a"
yolo_folder1 = r"C:\Users\michael\Desktop\f_proj\YOLO_test_resultsoriginal\predict\labels"
hs_folder1   = r"C:\Users\michael\Desktop\f_proj\300min\labels"
w1, h1 = 640, 480

gt1    = load_xml_ground_truths(gt_folder1, w1, h1, class_mapping={"drone":0})
yolo1  = load_yolo_labels(yolo_folder1,    normalize=False, img_width=w1, img_height=h1)
hs1    = load_yolo_labels(hs_folder1,      normalize=True,  img_width=w1, img_height=h1)

# --- Dataset 2 (new set: YOLO-format GT, 416×416) ---
gt_folder2      = r"C:\Users\michael\Desktop\f_proj\bird_output\labels"
yolo_folder2    = r"C:\Users\michael\Desktop\f_proj\bird_output\YOlOres\predict\labels"
hs_folder2      = r"C:\Users\michael\Desktop\f_proj\birdsdetections\labels"
w2, h2 = 416, 416

gt2   = load_yolo_labels(gt_folder2,    normalize=False, img_width=w2, img_height=h2)
for image_name, bboxes in gt2.items():
    for bbox in bboxes:
        if len(bbox) == 6:
            # format [prefix, class_id, x_c, y_c, w, h]
            bbox[1] = 1 - int(bbox[1])
        else:
            # format [class_id, x_c, y_c, w, h]
            bbox[0] = 1 - int(bbox[0])
yolo2= load_yolo_labels(yolo_folder2,   normalize=False, img_width=w2, img_height=h2)
hs2   = load_yolo_labels(hs_folder2,    normalize=True,  img_width=w2, img_height=h2)

# --- Per-dataset metrics ---
metrics = {}
metrics["YOLO_DS1"] = evaluate_detections(gt1, yolo1, w1, h1)
metrics["PIPE_DS1"] = evaluate_detections(gt1, hs1,   w1, h1)
metrics["YOLO_DS2"] = evaluate_detections(gt2, yolo2, w2, h2)
metrics["PIPE_DS2"] = evaluate_detections(gt2, hs2,   w2, h2)

# --- Combined across both datasets ---
combined = {}
combined["YOLO_Overall"] = combine_metrics(metrics["YOLO_DS1"], metrics["YOLO_DS2"])
combined["PIPE_Overall"] = combine_metrics(metrics["PIPE_DS1"], metrics["PIPE_DS2"])

    # --- Print everything ---

# --- Overall metrics across both datasets ---

for key, m in combined.items():
    prec_low, prec_high = m["Precision CI"]
    rec_low,  rec_high  = m["Recall CI"]
    print(f"\n-- {key} --")
    print(f" Precision: {m['Precision']:.3f} [{prec_low:.3f}, {prec_high:.3f}]")
    print(f" Recall:    {m['Recall']:.3f} [{rec_low:.3f}, {rec_high:.3f}]")
    print(f" F1-Score:  {m['F1-Score']:.3f}")
    print(f" Mean IoU:  {m['Mean IoU']:.3f}")
    print(f" TP={m['True Positives']}, FP={m['False Positives']}, FN={m['False Negatives']}")


-- YOLO_Overall --
 Precision: 0.890 [0.841, 0.926]
 Recall:    0.748 [0.691, 0.798]
 F1-Score:  0.813
 Mean IoU:  0.797
 TP=187, FP=23, FN=63

-- PIPE_Overall --
 Precision: 0.896 [0.849, 0.929]
 Recall:    0.824 [0.772, 0.866]
 F1-Score:  0.858
 Mean IoU:  0.791
 TP=206, FP=24, FN=44


#### How many images with drones were detected

In [ ]:

def image_detection_coverage(ground_truths, detections,
                             img_w, img_h, iou_threshold=0.5):
    """
    Returns (num_images_with_at_least_one_correct_detection ,
             total_images_that_contain_a_drone)
    """
    def is_drone(box):
        
        return (box[1] if len(box) == 6 else box[0]) == 0

    total = 0
    found = 0

    for img_name, gt_bboxes in ground_truths.items():

        # keep only drone GT boxes
        gt_drones = [gt for gt in gt_bboxes if is_drone(gt)]
        if not gt_drones:
            continue                      # image doesn’t contain a drone at all
        total += 1

        # candidate predictions for this image
        preds = detections.get(img_name, [])

        # de-duplicate predictions (ignore prefix) & keep only drones
        seen, pred_drones = set(), []
        for p in preds:
            if not is_drone(p):
                continue
            key = tuple(p[1:]) if len(p) == 6 else tuple(p)
            if key not in seen:
                seen.add(key)
                pred_drones.append(p)

        # does any pred match *any* GT with IoU ≥ threshold?
        hit = False
        for pred in pred_drones:
            pred_xy = yolo_to_xyxy(pred, img_w, img_h)
            for gt in gt_drones:
                gt_xy = yolo_to_xyxy(gt, img_w, img_h)
                if calculate_iou(pred_xy, gt_xy) >= iou_threshold:
                    hit = True
                    break
            if hit:
                break

        if hit:
            found += 1

    return found, total

# ────────────────────────────────────────────────────────────────
#  Overall “at least one drone found” coverage 
# ────────────────────────────────────────────────────────────────
yolo_hit_ds1,  yolo_tot_ds1  = image_detection_coverage(gt1,  yolo1, w1, h1)
yolo_hit_ds2,  yolo_tot_ds2  = image_detection_coverage(gt2,  yolo2, w2, h2)
pipe_hit_ds1,  pipe_tot_ds1  = image_detection_coverage(gt1,  hs1,  w1, h1)
pipe_hit_ds2,  pipe_tot_ds2  = image_detection_coverage(gt2,  hs2,  w2, h2)

yolo_hit_total  = yolo_hit_ds1  + yolo_hit_ds2
yolo_tot_total  = yolo_tot_ds1  + yolo_tot_ds2
pipe_hit_total  = pipe_hit_ds1  + pipe_hit_ds2
pipe_tot_total  = pipe_tot_ds1  + pipe_tot_ds2

print("\n=== Total per-image drone-found coverage ===")
print(f"YOLO_Overall: {yolo_hit_total}/{yolo_tot_total} images "
      f"({yolo_hit_total / yolo_tot_total * 100:.1f}%)")
print(f"PIPE_Overall: {pipe_hit_total}/{pipe_tot_total} images "
      f"({pipe_hit_total / pipe_tot_total * 100:.1f}%)")




=== Total per-image drone-found coverage ===
YOLO_Overall: 185/246 images (75.2%)
PIPE_Overall: 204/246 images (82.9%)


#### mAP calculation

In [ ]:
import numpy as np
from pathlib import Path
from collections import defaultdict

In [ ]:

def yolo_to_xyxy(bbox, img_w, img_h):
    """
    Convert YOLO format bbox [prefix, class, x_c, y_c, w, h] or [class, x_c, y_c, w, h]
    to [x1, y1, x2, y2] format (absolute coordinates).
    """
    if len(bbox) == 6:  # with prefix
        prefix, class_id, x_c, y_c, w, h = bbox
    else:  # without prefix
        class_id, x_c, y_c, w, h = bbox
        
    # Check if coordinates are normalized
    if all(0 <= val <= 1 for val in [x_c, y_c, w, h]):
        # Convert normalized to absolute
        x_c, w = x_c * img_w, w * img_w
        y_c, h = y_c * img_h, h * img_h
        
    # Convert center+dimensions to top-left, bottom-right
    x1 = x_c - w / 2
    y1 = y_c - h / 2
    x2 = x_c + w / 2
    y2 = y_c + h / 2
    
    return [x1, y1, x2, y2]

In [ ]:
def calculate_iou(box1, box2):
    """
    Calculate IoU between two boxes in [x1, y1, x2, y2] format.
    """
    # Get coordinates of intersection
    x1_inter = max(box1[0], box2[0])
    y1_inter = max(box1[1], box2[1])
    x2_inter = min(box1[2], box2[2])
    y2_inter = min(box1[3], box2[3])
    
    # Calculate area of intersection
    width_inter = max(0, x2_inter - x1_inter)
    height_inter = max(0, y2_inter - y1_inter)
    area_inter = width_inter * height_inter
    
    # Calculate area of both boxes
    area_box1 = (box1[2] - box1[0]) * (box1[3] - box1[1])
    area_box2 = (box2[2] - box2[0]) * (box2[3] - box2[1])
    
    # Calculate IoU
    area_union = area_box1 + area_box2 - area_inter
    iou = area_inter / area_union if area_union > 0 else 0
    
    return iou

In [ ]:
def load_yolo_labels_with_conf(label_folder, normalize=False, img_width=640, img_height=480):
    """
    Load YOLO format labels while keeping confidence values.
    Returns dict image_name -> list of [prefix, class_id, x_c, y_c, w, h, conf]
    """
    labels = {}
    for label_file in Path(label_folder).glob("*.txt"):
        image_name = label_file.stem
        with open(label_file) as f:
            bboxes = []
            for line in f:
                parts = line.strip().split()
                if not parts:
                    continue

                # classify by length
                if len(parts) == 7:
                    # prefix, class, x,y,w,h,conf
                    prefix = parts[0]
                    class_id, x_c, y_c, w, h, conf = map(float, parts[1:])
                elif len(parts) == 6:
                    # could be [class,x,y,w,h,conf] or [prefix,class,x,y,w,h]
                    try:
                        # try interpret last as conf
                        class_id = float(parts[0])
                        x_c, y_c, w, h, conf = map(float, parts[1:])
                        prefix = None
                    except ValueError:
                        # then it's prefix + 5 coords, no conf
                        prefix = parts[0]
                        class_id, x_c, y_c, w, h = map(float, parts[1:])
                        conf = 0.0
                elif len(parts) == 5:
                    # class, x,y,w,h (no conf)
                    prefix = None
                    class_id, x_c, y_c, w, h = map(float, parts)
                    conf = 0.0
                else:
                    # skip any other format
                    continue

                # optional renormalization
                if normalize and any(v > 1 for v in (x_c, y_c, w, h)):
                    x_c /= img_width
                    y_c /= img_height
                    w   /= img_width
                    h   /= img_height

                bboxes.append([prefix, class_id, x_c, y_c, w, h, conf])
        labels[image_name] = bboxes
    return labels


In [ ]:

def filter_drone_detections(detection_dict, drone_class_id=0):
    """
    Filter detection dictionary to only include drone detections.
    """
    filtered_dict = {}
    for img, dets in detection_dict.items():
        drone_dets = []
        for det in dets:
            # Format is [prefix, class_id, x, y, w, h, conf]
            if det[1] == drone_class_id:
                drone_dets.append(det)
        if drone_dets:
            filtered_dict[img] = drone_dets
    return filtered_dict

In [ ]:

def filter_drone_ground_truth(gt_dict, drone_class_id=0):
    """
    Filter ground truth dictionary to only include drone annotations.
    """
    filtered_dict = {}
    for img, gts in gt_dict.items():
        drone_gts = []
        for gt in gts:
            # Format could be [prefix, class_id, x, y, w, h] or [class_id, x, y, w, h]
            cls = gt[1] if len(gt) == 6 else gt[0]
            if cls == drone_class_id:
                drone_gts.append(gt)
        if drone_gts:
            filtered_dict[img] = drone_gts
    return filtered_dict

In [ ]:
def voc_map_50_drone_focused(gt_dict, det_dict, img_sizes, iou_thresh=0.5, drone_class_id=0):
    """
    Compute Pascal VOC 2007 11-point mAP@0.5 focusing only on drone detections.
    """
    drone_gt = filter_drone_ground_truth(gt_dict, drone_class_id)
    drone_det = filter_drone_detections(det_dict, drone_class_id)
    
    # Count ground truth positives
    npos = sum(len(gts) for gts in drone_gt.values())
    
    # Gather all predictions
    all_preds = []
    for img, preds in drone_det.items():
        for p in preds:
            *bbox, conf = p
            all_preds.append((img, bbox, conf))
    
    if not all_preds:
        return 0.0
        
    all_preds.sort(key=lambda x: x[2], reverse=True)
    
    claimed = {img: np.zeros(len(drone_gt.get(img, [])), dtype=bool)
               for img in drone_gt}
    tp = np.zeros(len(all_preds))
    fp = np.zeros(len(all_preds))
    
    for idx, (img, bbox_p, conf) in enumerate(all_preds):
        w, h = img_sizes[img]
        best_iou = 0
        best_j = -1
        p_xy = yolo_to_xyxy(bbox_p, w, h)
        
        if img in drone_gt:
            for j, gt in enumerate(drone_gt[img]):
                if claimed[img][j]:
                    continue
                g_xy = yolo_to_xyxy(gt, w, h)
                iou = calculate_iou(p_xy, g_xy)
                if iou > best_iou:
                    best_iou, best_j = iou, j
           
        if best_iou >= iou_thresh:
            claimed[img][best_j] = True
            tp[idx] = 1
        else:
            fp[idx] = 1
                
    cum_tp = np.cumsum(tp)
    cum_fp = np.cumsum(fp)
    rec = cum_tp / float(npos) if npos > 0 else np.zeros_like(cum_tp)
    prec = cum_tp / np.maximum(cum_tp + cum_fp, np.finfo(np.float64).eps)
    
    recall_levels = np.linspace(0, 1, 11)
    ap = 0.0
    for rl in recall_levels:
        p_at_rl = prec[rec >= rl]
        ap += np.max(p_at_rl) if p_at_rl.size else 0.0
    ap /= 11.0
    
    return ap

In [ ]:

def calculate_bird_false_positive_rate(gt_dict, det_dict, img_sizes, drone_class_id=0, bird_class_id=1):
    """
    Calculate false positive rate on bird-only images.
    """
    bird_only_images = set()
    for img, gts in gt_dict.items():
        has_drone = any((gt[1] if len(gt)==6 else gt[0])==drone_class_id for gt in gts)
        has_bird  = any((gt[1] if len(gt)==6 else gt[0])==bird_class_id  for gt in gts)
        if has_bird and not has_drone:
            bird_only_images.add(img)
    
    if not bird_only_images:
        return 0.0
    
    fp_count = 0
    for img in bird_only_images:
        if img in det_dict:
            for det in det_dict[img]:
                if det[1] == drone_class_id:
                    fp_count += 1
                    break
    
    return fp_count / len(bird_only_images)


In [26]:

DRONE_CLASS = 0
BIRD_CLASS  = 1

# Paths to your label folders
yolo_folder1 = r"C:\Users\michael\Desktop\f_proj\YOLOnoconf\predict\labels"
hs_folder1   = r"C:\Users\michael\Desktop\f_proj\mAPFixed\labels"
yolo_folder2 = r"C:\Users\michael\Desktop\f_proj\YOLOnoconfbird\predict\labels"
hs_folder2   = r"C:\Users\michael\Desktop\f_proj\noconfbirds\labels"

# Image sizes
w1, h1 = 640, 480
w2, h2 = 416, 416

# Load labels
yolo1_conf = load_yolo_labels_with_conf(yolo_folder1, normalize=False, img_width=w1, img_height=h1)
hs1_conf   = load_yolo_labels_with_conf(hs_folder1,   normalize=True,  img_width=w1, img_height=h1)
yolo2_conf = load_yolo_labels_with_conf(yolo_folder2, normalize=False, img_width=w2, img_height=h2)
hs2_conf   = load_yolo_labels_with_conf(hs_folder2,   normalize=True,  img_width=w2, img_height=h2)

# Image size dicts
img_sizes_1 = {img: (w1, h1) for img in set(list(yolo1_conf)+list(hs1_conf))}
img_sizes_2 = {img: (w2, h2) for img in set(list(yolo2_conf)+list(hs2_conf))}
img_sizes_combined = {**img_sizes_1, **img_sizes_2}

# Prepare sets
sets = {
    "YOLO_DS1": (gt1, yolo1_conf, img_sizes_1),
    "PIPE_DS1": (gt1, hs1_conf,   img_sizes_1),
    "YOLO_DS2": (gt2, yolo2_conf, img_sizes_2),
    "PIPE_DS2": (gt2, hs2_conf,   img_sizes_2),
}

results = {}
for name, (gt, dets, sizes) in sets.items():
    drone_ap      = voc_map_50_drone_focused(gt, dets, sizes, drone_class_id=DRONE_CLASS)
    bird_fp_rate  = calculate_bird_false_positive_rate(
        gt, dets, sizes, drone_class_id=DRONE_CLASS, bird_class_id=BIRD_CLASS
    )
    results[name] = {
        "drone_ap":     drone_ap,
        "bird_fp_rate": bird_fp_rate
    }

# Combined
for kind in ("YOLO", "PIPE"):
    combined_gt   = {**gt1, **gt2}
    combined_dets = {**sets[f"{kind}_DS1"][1], **sets[f"{kind}_DS2"][1]}
    drone_ap      = voc_map_50_drone_focused(
        combined_gt, combined_dets, img_sizes_combined, drone_class_id=DRONE_CLASS
    )
    bird_fp_rate  = calculate_bird_false_positive_rate(
        combined_gt, combined_dets, img_sizes_combined,
        drone_class_id=DRONE_CLASS, bird_class_id=BIRD_CLASS
    )
    results[f"{kind}_Overall"] = {
        "drone_ap":     drone_ap,
        "bird_fp_rate": bird_fp_rate
    }

# Print results

for key in ["YOLO_Overall", "PIPE_Overall"]:
    res = results[key]
    print(f"\n======== {key} ========")
    print(f" mAP@0.5:        {res['drone_ap']:.3f}")
    



======== YOLO_Overall ========
 mAP@0.5:        0.536

======== PIPE_Overall ========
 mAP@0.5:        0.536
